# Count Von Count's Twitter:
Notebook by Colin A. Stewart (colinstewart@engineering.ucsb.edu) 12-20-17
#### How high can The Count (from Sesame Street) count on twitter if he has a certain character limit, his numbers are written out in Latin letters, and he ends each number enthusiastically "!" ?

Puzzle from fivethirtyeight.com : https://fivethirtyeight.com/features/please-help-me-i-have-a-number-problem/

Generic Initiallize:

In [1]:
import math
# general math functions
import datetime
# system time & date etc.
import numpy as np
# numerical computation (sum, ave, random etc.)
import matplotlib.pyplot as plt
# basic plotting ability 
import seaborn as sns
# Makes plots pretty & allows you to keep plot settings consistent
import pandas as pd
# Data management & analysis 
%matplotlib inline
# When you run a cell, displays contents within the cell, like Mathematica
%config InlineBackend.figure_format = 'pdf'
# When it displays the contents in the cell, does so as a pdf

### Units lists names for 1-19, with the {# : "name"}

In [2]:
Units = {0:"",1:"one",2:"two",3:"three",4:"four",5:"five",6:"six",7:"seven",8:"eight",9:"nine",10:"ten",11:"eleven",12:"twelve",13:"thirteen",14:"fourteen",15:"fifteen",16:"sixteen",17:"seventeen",18:"eighteen",19:"nineteen"}

### Tens lists names for 20-90, with the {#/10 : "name"}

In [3]:
Tens = {2:"twenty",3:"thirty",4:"fourty",5:"fifty",6:"sixty",7:"seventy",8:"eighty",9:"ninety"}

### Suffixes lists names for 100, 1000, 10^6 - 10^45, with the {log(#) : "name"}

In [4]:
Suffixes = {0:"",3:"thousand",6:"million",9:"billion",12:"trillion",15:"quadrillion",18:"quintillion",21:"sextillion",24:"septillion",27:"octillion",30:"nonillion",33:"decillion",36:"undecillion",39:"duodecillion",42:"tredecillion",45:"quattuordecillion"}

### I. NumberNamer function enumerates an input positive integer from Arabic numerals to Latin letters
For a given number, we break it down into sets- or groups of 100's (3 digits)

Ex: 9,876,543,210 is:  [3rd set = 9],[2nd set = 876],[1st set = 543],[0th set = 210]

We then cycle through each set, starting with the largest.  We'll enumerate the set, and then subtract it from the running total and loop back through until all sets have been enumerated.

In [73]:
def NumberNamer(Arabic):
    # Store input number as double to 'running', which we'll subtract from as we enumerate it into string form
    # Then check that it's a positive integer & not too big for this script to handle
    running=float(Arabic)
    if ((running<=0)| (not ((running).is_integer()))):
        print("Invalid Input")
        return;
    powers = int(math.log10(running))
    if (powers>47):
        print("Input too big! I'll have to add to the 'Suffixes' Dictionary.")
        return;
    elif (running>3*10**22):
        print("Input too big! I think I'll have to change how variables are handled in memory.")
        return;
    # 'output' will hold the string to be returned, as we enumerate it set by set
    output = ""
    sets = powers//3
    
    # Go through each set, ie. group of 100s 
    while (sets>=0) & (running>0):
        powers = int(math.log10(running))
        sets = powers//3
        
        # If input number >999, take the greatest set.  If not, take the 0th set, ie. hundreds-tens-units 
        # Store the value of the set as 'setrun', and then subtract from this value as you enumerate the set into 'output'
        if (powers>2):
            setrun = running//(10**(sets*3))
        else:
            setrun = running
        
        # enumerate hundreds place of the running set 'setrun'
        if (setrun//100)>0:
            output = output+Units[setrun//100]+" hundred"
            # remove the part of the set you've just enumerated
            setrun-=(setrun//100)*100
            # if there's more after the hundreds place, put a space inbetween 
            if setrun>0:
                output+=" "
        # enumerate tens place of the running set 
        if (setrun//10)>1:
            output = output+Tens[setrun//10]
            setrun-=(setrun//10)*10
            if setrun>0:
                output+=" "   
        # enumerate units place of the running set 
        if (setrun<20)&(setrun>0):
            output = output+Units[setrun]
            setrun=0
        # if the running set is greater than the 0th set, ie. hundreds, add the set appendix 
        if (powers>2):
            output = output+" "+Suffixes[sets*3]
            if (running-(running//(10**(sets*3)))*(10**(sets*3)))>0:
                output+=" "
        
        # remove the whole set we just enumerated from the running value
        running-=(running//(10**(sets*3)))*(10**(sets*3))
       
        # re-evaluate the remainder of running value to be enumerated, restart loop
        if running>0:
            powers = int(math.log10(running))
        else:
            break;
        sets = powers//3
        


    # Say it with enthusiasm!  Also Capitallize  #
    output=output+"!"
    
    return output;

### II. TwitterCountUp function takes a starting value and character limit
It counts up from the starting value, enumerating each time in Latin letters, until it reaches the character limit.  Since twitter character limits allow you to count beyond 1 trillion, however, we'll probably need a smarter, faster way to reach the limit.

In [74]:
def TwitterCountUp(startval,CharLimit):
    # Print the time at start, and before each 100,000 enumerations so I can see how fast this runs
    print(str(datetime.datetime.now()))
    n = startval
    while len(NumberNamer(n))<= CharLimit:
        if ((n%100000)==0):
            print (n," = ",NumberNamer(n))
            print(str(datetime.datetime.now()))
        n+=1
    print (n," = ",NumberNamer(n)," =",len(NumberNamer(n)),"chars")
    return n;

### III. More smarter: let's find the different possible set lengths when enumerated, as well as the lengths of the suffixes separating them.
Make a list of the character lengths of all numbers 1-999 (all possible sets).  Remember to subtract 1 from NumberNamer results, -'!'.  We'll then make a list of the local maxima of character lengths (e.g. 3 = "Three" = 5 characters, while the next number 4 = "Four" = 4 characters, so "Three" is a local maximum).  

This is important because we want to achieve the highest number possible, so if we have enough characters to reach a local maximum, we should keep counting up until just before the next maximum.  These are 'free' numbers with no additional character penalty but higher numerical value.

When we join the sets together into one number, we'll connect them using Suffixes, so we'll make a list of the character length of all the suffixes (+2 for the surrounding spaces).

In [75]:
# The list of character lengths of all sets stored at 'SetLen'
SetLen=[]
for n in range(1,1000):
    SetLen.append(int(len(NumberNamer(n)))-1)
# print("Unique Values of Set Length = \n",list(set(SetLen)),"\n")


# If you wanna plot character length vs. set value, un-comment this below.  It's kinda neat I guess; this numerical pattern is just a consequence of the English language, so I feel like it's rather arbitrary.
# plt.scatter(range(1,1000),SetLen,label="Set Character Length vs. Numerical Value")


# The list of local maxima of character lengths stored at 'SetLenMax', and their corresponding numerical values stored at 'SetLenMaxVals'
print("Local Maxima of Set Length (SetLenMax) = ")
SetLenMax=[SetLen[1]]
SetLenMaxVals=[1]
print("Value = ",1,"Characters = ",3)
for n in range(2,int(len(SetLen))):
    if int(SetLen[n])>int(SetLenMax[-1]):
        SetLenMax.append((SetLen[n]))
        SetLenMaxVals.append(n+1)
        print("Value = ",n+1,"  Characters = ",SetLen[n])
        

# The list of character lengths of all suffixes w/ spaces stored at 'SuffixLen'
SuffixLen=[]
for n in range(1,16):
    SuffixLen.append(int(len(Suffixes[n*3]))+2)
# Insert 0 at beginning, since the 0th set has no suffix
SuffixLen.insert(0,0)
print("\nSuffix Lengths with spaces=",SuffixLen,"\n")

Local Maxima of Set Length (SetLenMax) = 
Value =  1 Characters =  3
Value =  3   Characters =  5
Value =  11   Characters =  6
Value =  13   Characters =  8
Value =  17   Characters =  9
Value =  21   Characters =  10
Value =  23   Characters =  12
Value =  73   Characters =  13
Value =  101   Characters =  15
Value =  103   Characters =  17
Value =  111   Characters =  18
Value =  113   Characters =  20
Value =  117   Characters =  21
Value =  121   Characters =  22
Value =  123   Characters =  24
Value =  173   Characters =  25
Value =  323   Characters =  26
Value =  373   Characters =  27

Suffix Lengths with spaces= [0, 10, 9, 9, 10, 13, 13, 12, 12, 11, 11, 11, 13, 14, 14, 19] 



### IV. We need to mathematically find the answer
The answer value should be as high as possible while still guaranteeing that all numbers up to and including itself are at or shorter than the character limit.  We'll write up the following steps so that we can solve this problem programatically for any arbitrary character limit:

**0)** Let's use the longest possible set value as 'filler': the absolute maximum 373 = "Three Hundred Seventy Three" (chars=27, see part III).  We can then think about building our number right to left.  Start with "!" (char=1) and then add each set consecutively, separated by each suffix +2 (for spaces). 

>**Ex:**  &ensp;  Building This Way &larr; ... + [1st set] + [1st Suffix] + [0th set] + !

**1)** Build a number composed of only filler sets and suffixes until you can't add more without passing the character limit.  If you've hit the character limit exactly with filler sets- nice job!  If not, continue 

**2)** Then, we'll take our remaining characters and add the highest set that first _hits or just goes over_ our character limit. 

**3)** Next, we're going to do a trick to 'squeeze out' an extra two characters we can use to count to higher numbers.  Count down on the 0th filler set by 1 (373&rarr;372) to go from (chars= 27&rarr;25).  We can then use these extra characters on one of the higher sets that has more numerical value than the 0th set.  Note from the example below that we can only use this trick once (we can't lower our 0th set by multiple values & use the additional extra characters to raise a later set by multiple values.    

>[max],373,373   **&rarr;**   
[max+2chars],373,372   **&rarr;**    
[max+2chars],373,37**3**  **&rarr;**  hits the character limit

**4)** Check your character limit- if you were over before & now you're right at the limit with the characters we squeezed out of the 0th set- nice job!  If you're still over, we'll make up for the number of characters over the limit by lowering the second to last set, the highest filler set (before we take away from it).  Replace the second to last set with the highest value you can within the character limit.  To do this, look at your character length without the length of the second to last set.  Find out how many characters you have left till you hit the limit.  Then find the first local maximum set value longer than this.  Then count down in set values until you hit your remaining characters.  

**5)** Put all your sets together, and double check using the NumberNamer that you're in your character limit.  Nice job!

In [108]:
def TwitterCountSmart(CharLimit):
    #We'll be storing our resultant number as a list of the sets (integers) in 'output' 
    output=[]
    #And we'll store the character count of the output when enumerated as 'outLen"
    outLen=0
    
    #Remember to work right to left- start with the '!'
    outLen=1 #the '!' is not represented in our 'output' list of integer sets, but we'll account for it in the 'outLen' 
    if outLen==CharLimit:
        return output;
    
    
    #Step 1. Fit as many filler sets as we can, this # = 'sets'
    sets=0
    while outLen<CharLimit:
        output.insert(0,373) #373 is the filler set & we prepend each new set
        outLen=(outLen+27+SuffixLen[sets]) #27 chars in the filler set
        sets+=1
    #We may have gone above the character limit, so remove last filler set (index=0) if so
    if outLen>CharLimit:
        sets-=1
        output=output[1:]
        outLen-=(27+SuffixLen[sets])  
    #test if we hit the character limit exactly
    if outLen==CharLimit:
        return output;
    print("Step 1: ",output," chars=",outLen,"\n")
    
    
    
    #Step 2. Add another set as high as you can at, or just over, our character limit
    outLen+=SuffixLen[sets] #indexes from 0
    sets+=1
    n=0  #counter
    # Find the highest set we can add
    while SetLen[n]<(CharLimit-outLen):
        n+=1
    output.insert(0,n+1)
    outLen+=SetLen[n]
    print("Step 2: ",output," chars=",outLen,"\n")

  
    
    #Step 3. Subtract 1 from set 0 to squeeze out extra characters
    output[-1]-=1
    outLen-=2
    print("Step 3: ",output," chars=",outLen,"\n")
    
    #Step 4. Check length vs character limit & subtract from 2nd to last set to make up character deficit
    if outLen==CharLimit:
        return output;
    # We're replacing the 2nd to last set (last filler set, 27 chars) with a smaller set that will let us hit the charLimit
    charBudget=(CharLimit-(outLen-27)-1) #-1 because want to hit, not go over charLimit
    # Find the highest set we can add
    n=0 # counter
    print("charBudget=",charBudget,"\n")
    while SetLenMax[n]<=charBudget:
        # Try to find a local maximum that exactly fits your characer budget (must be the 1st one to do so- you don't get 'free numbers' because of the 0th set character squeezing trick)
        print("n=",n," SetLenMax=",SetLenMax[n]," SetLenMaxVals=",SetLenMaxVals[n],"\n")
        if SetLenMax[n]==charBudget:
            output[1]=SetLenMaxVals[n]
            outLen=(outLen-26+SetLenMax[n])
        # If there's not an exact fit, find the first local max to surpass budget, and then count backwards until you're within budget
        if SetLenMax[n]>charBudget:
            x=SetLenMaxVals[n] #counter
            while SetLen[x]>charBudget:
                x-=1
            output[1]=x
            outLen+=SetLen[x]
        if n==16:
            break;
        n+=1
    print("Step 4: ",output," chars=",outLen,"\n")
    
    #Step 5. Turn the list of integer sets in 'output' into a numerical string, and also NumberNamer that number & check the character
    stringout=""
    numberout=""
    for n in output:
        stringout+=str(n)
        numberout+=str(n)
        if n==output[-1]:
            break;
        stringout+=","    
    print("Result =",stringout,"\n")
    print(NumberNamer(int(numberout)),"\n")
    print("Characters =",len(NumberNamer(int(numberout))))
    
    return;


### V. Finally, let's test our script for the old character limit, and then find the answer for the new limit!

In [111]:
TwitterCountSmart(140)

Step 1:  [373, 373, 373, 373]  chars= 137 

Step 2:  [1, 373, 373, 373, 373]  chars= 150 

Step 3:  [1, 373, 373, 373, 372]  chars= 148 

charBudget= 18 

n= 0  SetLenMax= 3  SetLenMaxVals= 1 

n= 1  SetLenMax= 5  SetLenMaxVals= 3 

n= 2  SetLenMax= 6  SetLenMaxVals= 11 

n= 3  SetLenMax= 8  SetLenMaxVals= 13 

n= 4  SetLenMax= 9  SetLenMaxVals= 17 

n= 5  SetLenMax= 10  SetLenMaxVals= 21 

n= 6  SetLenMax= 12  SetLenMaxVals= 23 

n= 7  SetLenMax= 13  SetLenMaxVals= 73 

n= 8  SetLenMax= 15  SetLenMaxVals= 101 

n= 9  SetLenMax= 17  SetLenMaxVals= 103 

n= 10  SetLenMax= 18  SetLenMaxVals= 111 

Step 4:  [1, 111, 373, 373, 372]  chars= 140 

Result = 1,111,373,373,372 

one trillion one hundred eleven billion three hundred seventy three million three hundred seventy three thousand three hundred seventy two! 

Characters = 139


In [110]:
TwitterCountSmart(280)

Step 1:  [373, 373, 373, 373, 373, 373, 373]  chars= 254 

Step 2:  [101, 373, 373, 373, 373, 373, 373, 373]  chars= 281 

Step 3:  [101, 373, 373, 373, 373, 373, 373, 372]  chars= 279 

charBudget= 27 

n= 0  SetLenMax= 3  SetLenMaxVals= 1 

n= 1  SetLenMax= 5  SetLenMaxVals= 3 

n= 2  SetLenMax= 6  SetLenMaxVals= 11 

n= 3  SetLenMax= 8  SetLenMaxVals= 13 

n= 4  SetLenMax= 9  SetLenMaxVals= 17 

n= 5  SetLenMax= 10  SetLenMaxVals= 21 

n= 6  SetLenMax= 12  SetLenMaxVals= 23 

n= 7  SetLenMax= 13  SetLenMaxVals= 73 

n= 8  SetLenMax= 15  SetLenMaxVals= 101 

n= 9  SetLenMax= 17  SetLenMaxVals= 103 

n= 10  SetLenMax= 18  SetLenMaxVals= 111 

n= 11  SetLenMax= 20  SetLenMaxVals= 113 

n= 12  SetLenMax= 21  SetLenMaxVals= 117 

n= 13  SetLenMax= 22  SetLenMaxVals= 121 

n= 14  SetLenMax= 24  SetLenMaxVals= 123 

n= 15  SetLenMax= 25  SetLenMaxVals= 173 

n= 16  SetLenMax= 26  SetLenMaxVals= 323 

Step 4:  [101, 373, 373, 373, 373, 373, 373, 372]  chars= 279 

Result = 101,373,373,373,3

TypeError: object of type 'NoneType' has no len()

### Epilogue-
With the new character limit, we get numbers too large for NumberNamer to handle with the way I've currently written the script (I think it's a variable memory issue).  But using our mathematical / programmatic approach, we still get an answer.  We'll have to check once the Riddler Express results are out